# DRAC2022 Task 2 and 3 Experiment Notebook
* creator: Jungrae Cho (team: FindDR)
* created: November 25 2022

## 1. Import

In [ ]:
# refresh import changes
%load_ext autoreload
%autoreload 2

import albumentations as A
import cv2
import gc
import numpy as np
import os
import pandas as pd

from glob import glob
from matplotlib import pyplot as plt
from natsort import natsorted
from os import path
from PIL import Image
from pathlib import Path

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

from scipy.ndimage import gaussian_filter
from scipy.io import loadmat

from skimage import io, color
from sklearn.model_selection import StratifiedKFold, KFold

import torch
import torch.nn.functional as F
from torch import nn
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor, Resize

from tqdm import tqdm
import timm

## 2. Parameters(num_classes, batch_size, and seed) and Transforms

In [ ]:
# image size dict
img_size_dict = {
    "resnet50d":224,
    "efficientnet_b3a":320,
    "efficientnet_b3":300,
    "skresnext50_32x4d":288,
    "seresnext50d_32x4d":224
}

# parpameters
GPU_NUM = 1
BATCH_SIZE = 32
TASK_NUM = 1 # Task 2: 1, Task 3: 2
SEED = 42
EPOCHS = 20
MODEL_NAME = "resnet50d" # choose one from resnet50d, efficientnet_b3a, efficientnet_b3, skresnext50_32x4d, or seresnext50d_32x4d
IMG_SIZE = img_size_dict[MODEL_NAME]

experiment_name = f"timm-{MODEL_NAME}-drac2022-task{TASK_NUM+1}"
num_classes = 3 
original_height = IMG_SIZE
original_width = IMG_SIZE

print(experiment_name)

Download DRAC2022 dataset checkt out its directory structure.

Dataset structure should be like this:
```
./DRAC2022_Data_Set/
    ├── DRAC2022_Testing_Set
    │   ├── A. Segmentation
    │   │   └── 1. Original Images
    │   │       └── b. Testing Set
    │   ├── B. Image Quality Assessment
    │   │   └── 1. Original Images
    │   │       └── b. Testing Set
    │   └── C. Diabetic Retinopathy Grading
    │       └── 1. Original Images
    │           └── b. Testing Set
    └── DRAC2022_Training_Set
        ├── A. Segmentation
        │   ├── 1. Original Images
        │   │   └── a. Training Set
        │   └── 2. Groundtruths
        │       └── a. Training Set
        │           ├── 1. Intraretinal Microvascular Abnormalities
        │           ├── 2. Nonperfusion Areas
        │           └── 3. Neovascularization
        ├── B. Image Quality Assessment
        │   ├── 1. Original Images
        │   │   └── a. Training Set
        │   └── 2. Groundtruths
        └── C. Diabetic Retinopathy Grading
            ├── 1. Original Images
            │   └── a. Training Set
            └── 2. Groundtruths
```

In [ ]:
data_root = "DRAC2022_Data_Set/" # Specifiy directory of DRAC2022 dataset.

img_paths = glob(os.path.join(data_root, "*","*","*","*","*.png")) # path of images
seg_paths = glob(os.path.join(data_root, "*","*","*","*","*","*.png")) # path of labels
csv_paths = glob(os.path.join(data_root, "*","*","*","*.csv")) # path of labels

print(len(img_paths), len(csv_paths), len(seg_paths))
print(img_paths[0])
print(seg_paths[0])

In [ ]:
parser = img_paths[0].split("/")
task = parser[-4]
data_split = parser[-2]
file_name = parser[-1]
print(f"{task}, {data_split}, {file_name}")

tasks = []
data_splits = []
file_names =[]
for img_path in tqdm(img_paths):
    parser = img_path.split("/")
    task = parser[-4]
    data_split = parser[-2]
    file_name = parser[-1]
    tasks.append(task)
    data_splits.append(data_split)
    file_names.append(file_name)
tasks = sorted(list(set(tasks))) 
data_splits = sorted(list(set(data_splits)))
print("\n")
print(tasks)
print(data_splits)
print(len(file_names))

task_img_count = {kk:{k:0 for k in tasks} for kk in data_splits}

for img_path in tqdm(img_paths):
    parser = img_path.split("/")
    task = parser[-4]
    data_split = parser[-2]
    task_img_count[data_split][task] += 1
print("\n")
print(task_img_count)

In [ ]:
data_paths = {
    "train":[],
    "test":[]
}
for img_path in img_paths:
    if tasks[TASK_NUM] in img_path:
        if data_splits[0] in img_path:
            data_paths["train"].append(img_path)
        else:
            data_paths["test"].append(img_path)
            
for k, v in data_paths.items():
    print(k,":", len(v))
    data_paths[k] = natsorted(v)

In [ ]:
labels = pd.read_csv(csv_paths[TASK_NUM-1])

In [ ]:
labels.head()

In [ ]:
data_paths['train']

In [ ]:
def to_categorical(y, num_classes):
    return np.eye(num_classes, dtype='uint8')[y]

class DRAC2022ClassificationDataset(object):
    def __init__(self, data_paths, csv_path, transforms, mode, num_classes):
        self.mode = mode
        self.num_classes = num_classes
        self.imgs = data_paths[self.mode]
        self.labels = pd.read_csv(csv_path)
        self.transforms=transforms
        
    def __getitem__(self, idx):

        img = cv2.imread(self.imgs[idx],-1).astype(np.uint8)
        img = np.stack([img,img,img], -1)
        
        if self.mode == 'train':

            file_name = self.imgs[idx].split('/')[-1]
            if TASK_NUM == 1:
                label = labels[labels["image name"]==file_name]["image quality level"].values
            else:
                label = labels[labels["image name"]==file_name]["DR grade"].values


            if self.transforms is not None:
                transformed = self.transforms(image=img)
                img = transformed['image'] / 255

                img = torch.from_numpy(img).permute(2,1,0).float()
                label = torch.as_tensor(label, dtype=torch.int64)
#                 label = to_categorical(label,self.num_classes)
#                 label = torch.from_numpy(label).long()

            return img, label
       
        else:
            
            if self.transforms is not None:
                transformed = self.transforms(image=img)
                img = transformed['image'] / 255

                img = torch.from_numpy(img).permute(2,1,0).float()
                
            return img, _
        
    def __len__(self):
        return len(self.imgs)

In [ ]:
def get_transform(train):
    if train:
        aug = A.Compose([
            A.Resize(original_height, original_width,p=1),
            A.Normalize(p=0.1),    
            A.VerticalFlip(p=0.5),              
            A.RandomRotate90(p=0.5),
            A.HorizontalFlip(p=0.5)
        ])
    else:
        aug = A.Compose([
            A.Resize(original_width, original_height,p=1),
            A.Normalize(p=0.1)
        ])
    return aug

In [ ]:
dataset = DRAC2022ClassificationDataset(data_paths, csv_paths[TASK_NUM-1], get_transform(True), 'train', num_classes)
dataset_test = DRAC2022ClassificationDataset(data_paths, csv_paths[TASK_NUM-1], get_transform(False), 'train', num_classes)

indices = torch.randperm(len(dataset)).tolist()
train_set = torch.utils.data.Subset(dataset, indices[:-50])
test_set = torch.utils.data.Subset(dataset_test, indices[-50:])

train_loader = DataLoader(train_set, shuffle=True, batch_size=BATCH_SIZE, num_workers=4, drop_last=True)
test_loader = DataLoader(test_set, shuffle=False, batch_size=1, num_workers=4)

In [ ]:
img, target = dataset.__getitem__(0)

In [ ]:
target.size(), img.size()

In [ ]:
target

In [ ]:
dataset.labels

In [ ]:
dataset_raw = DRAC2022ClassificationDataset(data_paths, csv_paths[TASK_NUM-1], None, 'train', num_classes)

In [ ]:
img, target = dataset_raw.__getitem__(0)

In [ ]:
target

In [ ]:
imgs = []
targets = []
perms = np.random.permutation(np.arange(dataset_raw.__len__()))
for j in range(0,3):
    for i in perms:
        img, target = dataset_raw.__getitem__(i)
        if target.item() == j:
            imgs.append(img)
            targets.append(target.item())
            break

In [ ]:
imgs

In [ ]:
targets

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(14,5))
# plt.suptitle(f"{seg_labels[i]}", y=0.9)
for i, img in enumerate(imgs):
    ax[i].imshow(img, cmap='gray')
    ax[i].axis("off")
plt.subplots_adjust(wspace=0, hspace=0.1)
plt.show()

In [ ]:
plt.imshow(img)
plt.axis("off")
plt.show()

## 3. Define classification model

In [ ]:
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, models
from pytorch_lightning.core.lightning import LightningModule
from torchmetrics.functional import accuracy

class LitModel(LightningModule):
    def __init__(self, num_classes,
                 train_loader, val_loader, test_loader):
        super().__init__()
        self.num_classes = num_classes
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.test_loader = test_loader
        self.model = timm.create_model(MODEL_NAME, pretrained=True, num_classes=self.num_classes)
        
        self.save_hyperparameters()
        
    def to(self,device):
        self.model.to(device)

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.model(x)
        loss = nn.CrossEntropyLoss()(y_hat, y.squeeze(1))
        tensorboard_logs = {'train_loss': loss}
        return {'loss': loss, 'log': tensorboard_logs}

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)

    def train_dataloader(self):
        loader = self.train_loader
        return loader
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.model(x)
        loss = nn.CrossEntropyLoss()(y_hat, y.squeeze(1))
        y_hat = y_hat.max(1)[1]
        acc = accuracy(y_hat, y)
        return {'val_loss': loss, 'val_acc': acc}

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        avg_acc = torch.stack([x['val_acc'] for x in outputs]).mean()
        tensorboard_logs = {'val_loss': avg_loss, 'val_acc':avg_acc}
        return {'avg_val_loss': avg_loss, 'avg_val_acc': avg_acc,
                'log': tensorboard_logs, 'progress_bar': tensorboard_logs}

    def val_dataloader(self):
        loader = self.val_loader
        return loader
    
    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.model(x)
        loss = nn.CrossEntropyLoss()(y_hat, y.squeeze(1))
        y_hat = y_hat.max(1)[1]
        acc = accuracy(y_hat, y)
        return {'test_loss': loss, 'test_acc': acc}

    def test_epoch_end(self, outputs):
        avg_loss = torch.stack([x['test_loss'] for x in outputs]).mean()
        avg_acc = torch.stack([x['test_acc'] for x in outputs]).mean()
        tensorboard_logs = {'test_loss': avg_loss, 'test_acc': avg_acc}
        return {'avg_test_loss': avg_loss, 'avg_test_acc': avg_acc, 'log': tensorboard_logs}

    def test_dataloader(self):
        loader = self.test_loader
        return loader

## 5. Set Random Seed and Create A Model Instance

In [ ]:
# seed everything
pl.seed_everything(SEED)

# callbacks
# early stopping
early_stop_callback = EarlyStopping(
   monitor='val_loss',
   min_delta=0.00,
   patience=20,
   verbose=True,
   mode='min'
)
logger = TensorBoardLogger('./lightning_logs', name=experiment_name)
ckpt = ModelCheckpoint(os.path.join('./lightning_logs',
                                    experiment_name,
                                    f'version_{logger.version}',
                                    'checkpoints'))

callbacks = [ckpt]

# define a model
model = LitModel(num_classes,
                 train_loader, test_loader, test_loader)

#     from torchsummary import summary
#     summary(model, (3, 256, 256))
device = torch.device(f'cuda:{GPU_NUM}') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

## 6. Train the Model

In [ ]:
# define trainer
trainer = pl.Trainer(gpus=[GPU_NUM], max_epochs=EPOCHS, callbacks=callbacks,
                    auto_lr_find=False)

# train the model
trainer.fit(model, train_loader, test_loader)

# test the model
trainer.test(model, test_loader)

# clear memory
gc.collect()
torch.cuda.empty_cache()

## 7. Test

In [ ]:
# load best model
best_root = f"./lightning_logs/{experiment_name}/"
print(best_root)

In [ ]:
best_ckpt = natsorted(glob(os.path.join(best_root,"*","*","*.ckpt")))[-1]
print(best_ckpt)
best_model = model.load_from_checkpoint(best_ckpt)

In [ ]:
"""
The implementation of Quadratic Weighted Kappa is from
https://blog.csdn.net/qq_35447659/article/details/107468778
"""

import numpy as np
from sklearn.metrics import roc_auc_score


def confusion_matrix(rater_a, rater_b, min_rating=None, max_rating=None):
    assert(len(rater_a) == len(rater_b))
    if min_rating is None:
        min_rating = min(rater_a + rater_b)
    if max_rating is None:
        max_rating = max(rater_a + rater_b)
    num_ratings = int(max_rating - min_rating + 1)
    conf_mat = [[0 for i in range(num_ratings)]
                for j in range(num_ratings)]
    for a, b in zip(rater_a, rater_b):
        conf_mat[a - min_rating][b - min_rating] += 1
    return conf_mat


def histogram(ratings, min_rating=None, max_rating=None):
    if min_rating is None:
        min_rating = min(ratings)
    if max_rating is None:
        max_rating = max(ratings)
    num_ratings = int(max_rating - min_rating + 1)
    hist_ratings = [0 for x in range(num_ratings)]
    for r in ratings:
        hist_ratings[r - min_rating] += 1
    return hist_ratings


def quadratic_weighted_kappa(rater_a, rater_b, min_rating=None, max_rating=None):
    rater_a = np.array(rater_a, dtype=int)
    rater_b = np.array(rater_b, dtype=int)
    assert(len(rater_a) == len(rater_b))
    if min_rating is None:
        min_rating = min(min(rater_a), min(rater_b))
    if max_rating is None:
        max_rating = max(max(rater_a), max(rater_b))
    conf_mat = confusion_matrix(rater_a, rater_b, min_rating, max_rating)
    num_ratings = len(conf_mat)
    num_scored_items = float(len(rater_a))

    hist_rater_a = histogram(rater_a, min_rating, max_rating)
    hist_rater_b = histogram(rater_b, min_rating, max_rating)

    numerator = 0.0
    denominator = 0.0

    for i in range(num_ratings):
        for j in range(num_ratings):
            expected_count = (hist_rater_a[i] * hist_rater_b[j] / num_scored_items)
            d = pow(i - j, 2.0) / pow(num_ratings - 1, 2.0)
            numerator += d * conf_mat[i][j] / num_scored_items
            denominator += d * expected_count / num_scored_items
    return 1.0 - numerator / denominator


if __name__ == "__main__":
    y_true = np.array([0, 0, 1, 2], dtype=int)
    y_pred = np.array([0, 0, 1, 0], dtype=int)
    y_scores = np.array([[0.55, 0.2, 0.25],
                         [0.8, 0.1, 0.1],
                         [0.1, 0.7, 0.2],
                         [0.8, 0.1, 0.1]],
                        dtype=np.float64)
    Kw = quadratic_weighted_kappa(y_true, y_pred)
    AUC = roc_auc_score(y_true, y_scores, average="macro", multi_class='ovo')
    print(Kw, AUC)

In [ ]:
test_iter = iter(test_loader)
best_model.eval()
img_list = []
y_true = []
y_pred = []
y_pred_score = []
for i in range(len(test_loader)):
    img, gt = next(test_iter)
    img_np = img.squeeze(0).permute(1,2,0).mul(255).detach().cpu().numpy().astype(np.uint8).copy()
    img_list.append(img_np)
    gt_np = gt.squeeze(0).numpy()
    with torch.no_grad():
        yhat = best_model(img)
        yhat_np = yhat.max(1)[1].detach().cpu().numpy()
        yhat_score = torch.softmax(yhat,-1).detach().cpu().numpy()
        y_true.append(gt_np[0])
        y_pred.append(yhat_np[0])
        y_pred_score.append(yhat_score[0])
y_true = np.array(y_true, dtype=int)
y_pred = np.array(y_pred, dtype=int)
y_pred_score = np.array(y_pred_score, dtype=float)

In [ ]:
np.set_printoptions(suppress=True)
y_pred_score[:3]

In [ ]:
Kw = quadratic_weighted_kappa(y_true, y_pred)
AUC = roc_auc_score(y_true, y_pred_score, average="macro", multi_class='ovo')
Kw, AUC

# 7. Export submission data

In [ ]:
# submission data generation
dataset_test_real = DRAC2022ClassificationDataset(data_paths, csv_paths[TASK_NUM-1], get_transform(False), 'test', num_classes)
test_real_loader = DataLoader(dataset_test_real, shuffle=False, batch_size=1, num_workers=4)

n_test_real = dataset_test_real.__len__()

best_model.eval()
case_list = []
class_list = []
P0_list = []
P1_list = []
P2_list = []
for i in range(n_test_real):
    img, _ = dataset_test_real.__getitem__(i)
    img = img.unsqueeze(0)
    
    case_name = dataset_test_real.imgs[i].split('/')[-1]
    
    with torch.no_grad():
        yhat = best_model(img)
        yhat_np = yhat.max(1)[1].detach().cpu().numpy()
        yhat_score = torch.softmax(yhat,-1).detach().cpu().numpy()

        case_list.append(case_name)
        class_list.append(yhat_np[0])
        P0_list.append(yhat_score[0,0])
        P1_list.append(yhat_score[0,1])
        P2_list.append(yhat_score[0,2])

submission_dict = {
    "case": case_list,
    "class": class_list,
    "P0": P0_list,
    "P1": P1_list,
    "P2": P2_list,
}

submission_df = pd.DataFrame(submission_dict, columns=['case', 'class', 'P0', 'P1', 'P2'])

submission_df.to_csv(f"{experiment_name}_TEAMNAME.csv", index=False)

# References
1. [Pytorch Lightning](https://github.com/Lightning-AI/lightning)
2. [pytorch-image-models;timm](https://github.com/rwightman/pytorch-image-models)
3. [DRAC2022 submission example](https://github.com/zhuanjiao2222/DRAC2022)
4. [DRAC2022 official web site](https://drac22.grand-challenge.org/)